In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [2]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0   309k      0 --:--:-- --:--:-- --:--:--  310k


In [3]:
pd.read_csv("datasets/pima_indians_imputed.csv", index_col=0).astype(float).to_csv("datasets/pima_indians_imputed.csv")

In [4]:
def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_accuracy(model, data_loader):
    correct = 0
    total = 0
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)

    accuracy = correct / total if total > 0 else 0
    print(f"Correct: {correct}, Total: {total}, Accuracy: {accuracy:.4f}")
    return accuracy


In [5]:
kb = KnowledgeBase("medical_config.yaml")

In [6]:
compute_accuracy(kb.predicates["Classifier"], kb.loaders[0])

Correct: 268, Total: 768, Accuracy: 0.3490


0.3489583333333333

In [7]:
kb.optimize(num_epochs=501, log_steps=100)

['∀ person.(((y == diabetes) -> Classifier(person)))', '∀ person.(((y == healthy) -> ~(Classifier(person))))']
Rule Outputs:  [tensor(0.3277, grad_fn=<RsubBackward1>), tensor(0.8468, grad_fn=<RsubBackward1>)]
Epoch 1/501, Loss: 0.4875742197036743

['∀ person.(((y == diabetes) -> Classifier(person)))', '∀ person.(((y == healthy) -> ~(Classifier(person))))']
Rule Outputs:  [tensor(0.5875, grad_fn=<RsubBackward1>), tensor(0.6722, grad_fn=<RsubBackward1>)]
Epoch 101/501, Loss: 0.3725748658180237

['∀ person.(((y == diabetes) -> Classifier(person)))', '∀ person.(((y == healthy) -> ~(Classifier(person))))']
Rule Outputs:  [tensor(0.6274, grad_fn=<RsubBackward1>), tensor(0.6935, grad_fn=<RsubBackward1>)]
Epoch 201/501, Loss: 0.34117352962493896

['∀ person.(((y == diabetes) -> Classifier(person)))', '∀ person.(((y == healthy) -> ~(Classifier(person))))']
Rule Outputs:  [tensor(0.6419, grad_fn=<RsubBackward1>), tensor(0.7084, grad_fn=<RsubBackward1>)]
Epoch 301/501, Loss: 0.32653164863586426



In [8]:
compute_accuracy(kb.predicates["Classifier"], kb.loaders[0])

Correct: 548, Total: 768, Accuracy: 0.7135


0.7135416666666666